<a href="https://colab.research.google.com/github/evainnocente/537-project/blob/main/Variant_calling_and_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

bcftools mpileup and call

In [ ]:
code for project

filter